In [7]:
import os
train_data_path = 'data/train/2025-04-11'
train_file_paths = list()
skip_files = ['stopwords']

for root, dirs, files in os.walk(train_data_path):
    for file in files:
        if file not in skip_files:
            train_file_paths.append(f'{train_data_path}/{file}')

print(train_file_paths)

['data/train/2025-04-11/1-相亲闪婚后，老公竟是千亿财阀.txt', 'data/train/2025-04-11/10-腹黑娇妻：厉少，人设崩坏中.txt', 'data/train/2025-04-11/100 小夫人一身反骨，大叔乖乖臣服.txt', 'data/train/2025-04-11/101 墨爷的闪婚小甜妻.txt', 'data/train/2025-04-11/102 流年不负你情深.txt', 'data/train/2025-04-11/13-帝少甜婚令：娇妻，乖一点.txt', 'data/train/2025-04-11/14-闪婚厚爱：病娇老公竟是千亿大佬.txt', 'data/train/2025-04-11/17-闪婚三年不见面，再遇我是他秘书.txt', 'data/train/2025-04-11/一夜惊喜：超级总裁喜当爹.txt', 'data/train/2025-04-11/一胎六宝：总裁爸比超凶猛.txt', 'data/train/2025-04-11/一胎双宝：季总你出局了.txt', 'data/train/2025-04-11/七年之痒，前夫不知道我重生了.txt', 'data/train/2025-04-11/七爷的罪妻.txt', 'data/train/2025-04-11/三宝联萌 大佬妈咪拽又飒.txt', 'data/train/2025-04-11/三宝联萌：大佬妈咪拽又飒.txt', 'data/train/2025-04-11/三年恋情曝光，我成了白月光的替身.txt', 'data/train/2025-04-11/不受宠？无所谓，杀疯豪门万人跪 (1~20章)-总裁、多重身份、傲慢霸气、复仇、男-有魅力的角色、浪漫、甜蜜、言情、大女主、甜宠.txt', 'data/train/2025-04-11/不当舔狗后，肥妻她美又飒！ (1~20章)-多重身份、离婚、豪门霸总、真假千金、追妻火葬场、破镜重圆、甜宠、打脸.txt', 'data/train/2025-04-11/乔秘书离职结婚，陆总醋疯了.txt', 'data/train/2025-04-11/九零团宠三岁半.txt', 'data/train/2025-04-11/二婚甜妻：祁少，正经点！.txt',

In [8]:
from src.util.read_file import read_file

encoding = 'utf-8'
train_novels = list[tuple]()

for file_path in train_file_paths:
    try:
        train_novels.append((file_path, read_file(file_path, encoding)))
    except Exception:
        print('Decode Error ', file_path)

with open('data/stopwords', encoding='utf-8') as _f:
    stopwords = _f.readlines()

print(str(train_novels[0])[:512], '...')
print(stopwords[:128])

Decode Error  data/train/2025-04-11/七爷的罪妻.txt
Decode Error  data/train/2025-04-11/三宝联萌 大佬妈咪拽又飒.txt
Decode Error  data/train/2025-04-11/大佬你的小娇娇甜翻了.txt
Decode Error  data/train/2025-04-11/星河灿烂皆因你.txt
Decode Error  data/train/2025-04-11/暖婚急诊：商医生，来盖个章！.txt
('data/train/2025-04-11/1-相亲闪婚后，老公竟是千亿财阀.txt', '\n第1章 来闪婚还摆架子\n书名： 相亲闪婚后，老公竟是千亿财阀 作者： 一板奶片 本章字数： 2164 更新时间： 2023-01-18 21:15:55\n\n“清溪，今天晚上你在外面吃个饭再回来了，没钱舅舅给你，你舅妈在发疯呢……”\n\n电话那头舅舅的话还没说完，又紧跟着传来舅妈尖锐的声音。\n\n“我发疯？你那贱种侄女今年26岁还在读书，不嫁人在我家吃我的用我的，还说我发疯？！现在公司遇到这么大的危机，我不管，你必须把她介绍给……”\n\n余清溪刚准备开口，电话匆忙地挂断了。\n\n她无声地叹了口气，看着自己刚发的奖学金，一份没动给舅舅转了过去。\n\n这样的情况，基本上每个星期都会出现。\n\n自从她妈妈去世后，她被舅舅接回家。因为舅妈不喜欢她，她只能在这个家小心翼翼地生活着，直到考上了文物修复学的研究生，她才将前些年欠舅舅的学费用奖学金还干净了。\n\n本想着，这个家无论如何她都得搬走了，苦于舅舅一直不同意，舅妈也因此一直跟他吵架。\n\n她唯一能想到能够正大光明搬走，舅舅不会担心的借口，就是结婚！ ...
['!\n', '"\n', '#\n', '$\n', '%\n', '&\n', "'\n", '(\n', ')\n', '*\n', '+\n', ',\n', '-\n', '--\n', '.\n', '..\n', '...\n', '......\n', '...................\n', './\n', '.一\n', '.数\n', '.日\n', '/\n', '//\n', '0\n', '1\n', '2\n', '

In [ ]:
from src.util.preprocess import split_words

train_seg_lists = list()

for novel in train_novels:
    train_seg_lists.append((novel[0], split_words(novel[1], stopwords)))
    
print(train_seg_lists[0])

Names: ['一板奶片', '清溪', '余清溪', '余清溪', '清溪', '余清溪', '裴奶奶', '裴奶', '余清溪', '裴奶奶', '裴', '余清溪', '裴', '余清溪', '裴奶奶', '裴奶奶', '余清溪', '余清溪', '余清溪', '裴奶', '上帝', '裴奶', '裴奶', '裴漠', '衍', '清溪', '裴', '裴漠', '衍神', '余清溪', '余清溪', '清溪', '裴漠', '衍', '清溪', '裴奶', '裴奶奶', '裴', '余清溪', '裴漠', '衍', '裴漠', '衍', '余晴雨', '裴漠', '衍', '裴', '余清溪', '余清溪', '裴漠', '衍身', '余清溪', '余清溪', '裴', '裴', '裴漠', '余清溪', '裴漠', '裴漠', '裴奶奶', '余清溪', '裴', '裴漠', '衍', '裴奶', '裴家', '裴漠衍', '裴氏', '裴奶奶', '裴漠', '衍', '余清溪', '余清溪', '裴漠', '衍', '余清溪', '裴奶', '余清溪', '余清溪', '清溪', '王总', '余清溪', '张晴', '余清溪', '裴奶奶', '裴奶', '裴漠', '衍', '裴漠', '威逼利', '余清溪', '余清溪', '裴', '裴漠衍', '裴', '余清溪', '裴', '裴漠', '衍', '余清溪', '余清溪', '裴', '余清溪', '余清溪', '张晴', '余雯雯', '清溪', '张晴满脸', '余清溪', '余氏', '余清溪', '清溪', '余氏', '余家', '张晴', '余清溪', '余氏', '张晴', '清溪', '王家能', '王', '余清溪', '张', '晴', '王', '余雯雯', '王', '余雯雯', '张', '晴', '余清溪', '王', '余雯', '余清溪', '黄花大', '张', '晴', '余清溪', '王总', '余雯', '王', '余清溪', '裴漠衍', '裴漠', '裴', '裴总', '裴漠', '衍', '一板奶片', '裴', '余清溪', '裴漠', '王', '余家', '凌慕', '余清溪', '王', '王', '王', '余雯雯', '余清溪'

In [ ]:
train_documents = [x[1] for x in train_seg_lists]

In [ ]:
from gensim import corpora

dictionary = corpora.Dictionary(train_documents)
corpus = [dictionary.doc2bow(x) for x in train_documents]

print(corpus[:16])

In [ ]:
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

num_topics = 10

lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    passes=64,
    random_state=32
)


cm = CoherenceModel(model=lda_model, texts=train_documents, coherence='u_mass')
coherence = cm.get_coherence()
print(f"Coherence={coherence}")

In [ ]:
pyLDAvis.display(gensimvis.prepare(lda_model, corpus, dictionary))

In [ ]:
# save model
lda_model.save('model/lda/lda_model.pkl')